# Firesmoke Data Conversion to IDX using OpenVisus

## Import necessary libraries, install them if you do not have them. This was developed in Python 3.9

In [ ]:
# Used to read/manipulate netCDF data
import xarray as xr

# Used to convert to .idx
from OpenVisus import *

# Used for numerical work
import numpy as np

# Used for processing netCDF time data
import datetime

# Used for interacting with OS file system (to get directory file names)
import os

# To load/save final sequence array to file
import pickle

# Used for resampling arrays to fit the same lat/lon grid
from scipy.interpolate import griddata

# for plotting
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Accessory, used to generate progress bar for running for loops
# from tqdm.notebook import tqdm
# import ipywidgets
# import jupyterlab_widgets
from tqdm import tqdm

## Get relevant directory paths

In [ ]:
# ******* THIS IS WHEN RUNNING FROM ATLANTIS.SCI **************
firesmoke_dir = "/usr/sci/cedmav/data/firesmoke"

# path to save idx file and data
idx_dir = "/usr/sci/scratch_nvme/arleth/idx/firesmoke"

In [ ]:
# get metadata of datasets, had to be obtained manually
ids = ["BSC18CA12-01", "BSC00CA12-01", "BSC06CA12-01", "BSC12CA12-01"]
start_dates = ["20210304", "20210304", "20210304", "20210303"]
end_dates = ["20231016", "20240210", "20231016", "20231015"]

id_dates = {ids[i]: {"start_date": start_dates[i], "end_date": end_dates[i]} for i in range(len(ids))}

## Gather information about the metadata of our files, since it is inconsistent file to file. We need to know what to normalize across all files.

### In particular:
#### 1. Count number of files there are per firesmoke directory.
#### 2. Determine maximum row,col dimension sizes for pm25 array.
#### 3. Determine maximum latitude longitude grid parameters.

In [ ]:
# List of all files that are available from UBC
successful_files = {id_: [] for id_ in ids}

# Variables to hold maxes, also to track the unique max values
max_ncols = {id_: 0 for id_ in ids}
max_nrows = {id_: 0 for id_ in ids}
ncols = {id_: set() for id_ in ids}
nrows = {id_: set() for id_ in ids}

# Max grid dimensions
max_grid_x = {id_: {"xorig": 0.0, "xcell": 0.0} for id_ in ids}
max_grid_y = {id_: {"yorig": 0.0, "ycell": 0.0} for id_ in ids}
xorigs = {id_: set() for id_ in ids}
xcells = {id_: set() for id_ in ids}
yorigs = {id_: set() for id_ in ids}
ycells = {id_: set() for id_ in ids}

for id_ in ids:
    # get list of netcdf file names for each dataset
    file_names = os.listdir(f'{firesmoke_dir}/{id_}/')
    
    # try opening each file, process only if it successfully opens
    for file in tqdm(file_names):
        # get file's path
        path = f'{firesmoke_dir}/{id_}/{file}'
        
        # keep track of which files successfully open
        try:
            # open the file with xarray
            ds = xr.open_dataset(path)
    
            # append file name to successful_files
            successful_files[id_].append(file)
    
            # update maxes accordingly
            # these *are* allowed to get mixed up between files right? in this case don't need to worry bout it
            max_ncols[id_] = max(max_ncols[id_], ds.NCOLS)
            max_nrows[id_] = max(max_nrows[id_], ds.NROWS)
    
            # these should not get mixed up between files right? or can they?
            # if they do get mixed up, wouldn't it be a ill-defined grid?
            # ref: https://stackoverflow.com/questions/18296755/python-max-function-using-key-and-lambda-expression
            max_grid_x[id_]["xorig"] = max(max_grid_x[id_]["xorig"], ds.XORIG, key=abs)
            max_grid_y[id_]["yorig"] = max(max_grid_y[id_]["yorig"], ds.YORIG, key=abs)
            max_grid_x[id_]["xcell"] = max(max_grid_x[id_]["xcell"], ds.XCELL, key=abs)
            max_grid_y[id_]["ycell"] = max(max_grid_y[id_]["ycell"], ds.YCELL, key=abs)
    
            # update sets
            ncols[id_].add(ds.NCOLS)
            nrows[id_].add(ds.NROWS)
            xorigs[id_].add(ds.XORIG)
            yorigs[id_].add(ds.YORIG)
            xcells[id_].add(ds.XCELL)
            ycells[id_].add(ds.YCELL)
            
        except:
            # netcdf file does not exist
            continue

# Sort datasets' lists of successful files so they're in order of date
for id_ in successful_files:
    successful_files[id_] = np.sort(successful_files[id_]).tolist()

In [ ]:
# Print the information for all ids
for id_ in ids:
    print(f'dataset: {id_}')
    print(f'Number of successful files: {len(successful_files[id_])}')
    print(f'Max cell sizes: max_ncols = {max_ncols[id_]} and max_nrows = {max_nrows[id_]}')
    print(f'Max xorig & xcell: {max_grid_x[id_]}')
    print(f'Max yorig & ycell: {max_grid_y[id_]}')
    print(f'ncols: {ncols[id_]}')
    print(f'nrows: {nrows[id_]}')
    print(f'xorigs: {xorigs[id_]}')
    print(f'yorigs: {yorigs[id_]}')
    print(f'xcells: {xcells[id_]}')
    print(f'ycells: {ycells[id_]}')
    print()

### Get latitude/longitude coordinates using the max values and non-max values AMONG ALL DATASETS, this is used for resampling during conversion

Luckily, all datasets have the same 'smaller' and 'larger' lat/lon grid parameters :)

In [ ]:
# get parameters for bigger lat/lon
max_xorig = max_grid_x[ids[0]]['xorig']
max_xcell = max_grid_x[ids[0]]['xcell']
max_yorig = max_grid_y[ids[0]]['yorig']
max_ycell = max_grid_y[ids[0]]['ycell']

# get arrays of bigger lat/lon grid
big_lon = np.linspace(max_xorig, max_xorig + max_xcell * (max_ncols[ids[0]] - 1), max_ncols[ids[0]])
big_lat = np.linspace(max_yorig, max_yorig + max_ycell * (max_nrows[ids[0]] - 1), max_nrows[ids[0]])

# get coordinates made of new lat/lon arrays
big_lon_pts, big_lat_pts = np.meshgrid(big_lon, big_lat)
big_tups = np.array([tup for tup in zip(big_lon_pts.flatten(), big_lat_pts.flatten())])

# get arrays of smaller lat/lon grid
sml_ds = xr.open_dataset(firesmoke_dir + "/BSC00CA12-01/dispersion_20210304.nc")
sml_lon = np.linspace(sml_ds.XORIG, sml_ds.XORIG + sml_ds.XCELL * (sml_ds.NCOLS - 1), sml_ds.NCOLS)
sml_lat = np.linspace(sml_ds.YORIG, sml_ds.YORIG + sml_ds.YCELL * (sml_ds.NROWS - 1), sml_ds.NROWS)

# get coordinates made of small lat/lon arrays
sml_lon_pts, sml_lat_pts = np.meshgrid(sml_lon, sml_lat)
sml_tups = np.array([tup for tup in zip(sml_lon_pts.flatten(), sml_lat_pts.flatten())])

## TESTING `resample_array` AND SCRIBBLES

### This is plotting the oiginal 381x1041 file

In [ ]:
# # Get the PM25 values, squeeze out empty axis
# vals = np.squeeze(sml_ds['PM25'].values)

# # Perform the interpolation
# arr = griddata(sml_tups, vals[15].flatten(), big_tups, method='cubic', fill_value=0)

# # Any values that are less than a given threshold, make it 0
# arr[arr < 1e-15] = 0

# # Reshape the result to match the new grid shape
# arr = arr.reshape((len(big_lat), len(big_lon)))

# arr = arr.astype(np.float32)

In [ ]:
# np.min(arr)

In [ ]:
# type(arr[0,0])

In [ ]:
# # Let's use matplotlib's imshow, since our data is on a grid
# # ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html

# # Initialize a figure and plot, so we can customize figure and plot of data
# # ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html
# # ref: https://scitools.org.uk/cartopy/docs/latest/getting_started/index.html
# my_fig, my_plt = plt.subplots(figsize=(15, 6), subplot_kw=dict(projection=ccrs.PlateCarree()))

# # Let's set some parameters to get the visualization we want
# # ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html

# # color PM25 values on a log scale, since values are small
# my_norm = "log" 
# # this will number our x and y axes based on the longitude latitude range
# my_extent = [np.min(sml_lon), np.max(sml_lon), np.min(sml_lat), np.max(sml_lat)]
# # ensure the aspect ratio of our plot fits all data, matplotlib can does this automatically
# my_aspect = 'auto'
# # tell matplotlib, our origin is the lower-left corner
# my_origin = 'lower'
# # select a colormap for our plot and the color bar on the right
# my_cmap = 'viridis'

# # create our plot using imshow
# plot = my_plt.imshow(arr, norm=my_norm, extent=my_extent, 
#           aspect=my_aspect, origin=my_origin, cmap=my_cmap)

# # draw coastlines
# my_plt.coastlines()

# # draw latitude longitude lines
# # ref: https://scitools.org.uk/cartopy/docs/latest/gallery/gridlines_and_labels/gridliner.html
# my_plt.gridlines(draw_labels=True)

# # add a colorbar to our figure, based on the plot we just made above
# my_fig.colorbar(plot,location='right', label='ug/m^3')

# # # Set x and y axis labels on our ax
# # my_plt.set_xlabel('Longitude')
# # my_plt.set_ylabel('Latitude')

# # Set title of our figure
# my_fig.suptitle('Ground level concentration of PM2.5 microns and smaller')

# # # Set title of our plot as the timestamp of our data
# # my_plt.set_title(f'{my_timestamp}')

# # Show the resulting visualization
# plt.show()

### This is visualizing the resampled version of array above, from 381x1041 -> 381x1081 grid

In [ ]:
# # Let's use matplotlib's imshow, since our data is on a grid
# # ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html

# # Initialize a figure and plot, so we can customize figure and plot of data
# # ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html
# # ref: https://scitools.org.uk/cartopy/docs/latest/getting_started/index.html
# my_fig, my_plt = plt.subplots(figsize=(15, 6), subplot_kw=dict(projection=ccrs.PlateCarree()))

# # Let's set some parameters to get the visualization we want
# # ref: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html

# # color PM25 values on a log scale, since values are small
# my_norm = "log" 
# # this will number our x and y axes based on the longitude latitude range
# my_extent = [np.min(big_lon), np.max(big_lon), np.min(big_lat), np.max(big_lat)]
# # ensure the aspect ratio of our plot fits all data, matplotlib can does this automatically
# my_aspect = 'auto'
# # tell matplotlib, our origin is the lower-left corner
# my_origin = 'lower'
# # select a colormap for our plot and the color bar on the right
# my_cmap = 'viridis'

# # create our plot using imshow
# plot = my_plt.imshow(arr_resamp, norm=my_norm, extent=my_extent, 
#           aspect=my_aspect, origin=my_origin, cmap=my_cmap, vmin=.00001, vmax=1)

# # draw coastlines
# my_plt.coastlines()

# # draw latitude longitude lines
# # ref: https://scitools.org.uk/cartopy/docs/latest/gallery/gridlines_and_labels/gridliner.html
# my_plt.gridlines(draw_labels=True)

# # add a colorbar to our figure, based on the plot we just made above
# my_fig.colorbar(plot,location='right', label='ug/m^3')

# # # Set x and y axis labels on our ax
# # my_plt.set_xlabel('Longitude')
# # my_plt.set_ylabel('Latitude')

# # Set title of our figure
# my_fig.suptitle('Ground level concentration of PM2.5 microns and smaller')

# # # Set title of our plot as the timestamp of our data
# # my_plt.set_title(f'{my_timestamp}')

# # Show the resulting visualization
# plt.show()

## Determine sequence of files to load later for IDX conversion

### First determine what hours are available in all datasets, from there we construct final sequence

In [ ]:
# for parsing time flags (TFLAG) from netcdf files
def parse_tflag(tflag):
    year = int(tflag[0] // 1000)
    day_of_year = int(tflag[0] % 1000)
    date = datetime.datetime(year, 1, 1) + datetime.timedelta(days=day_of_year - 1)

    time_in_day = int(tflag[1])
    hours = time_in_day // 10000
    minutes = (time_in_day % 10000) // 100
    seconds = time_in_day % 100

    full_datetime = datetime.datetime(year, date.month, date.day, hours, minutes, seconds)
    return full_datetime

In [ ]:
# get set of all available hours for each dataset using successful_files
id_sets = {id_: {} for id_ in ids}

for id_ in ids:    
    # get successful files to add all successful hours to set
    for file in tqdm(successful_files[id_]):
        # get file's path
        path = f'{firesmoke_dir}/{id_}/{file}'
        
        # open the file with xarray
        ds = xr.open_dataset(path)

        # add each available hour to successful_seq, store the index h, needed for idx conversion
        for h in range(ds.sizes["TSTEP"]):
            id_sets[id_][(file, parse_tflag(ds['TFLAG'].values[h][0]))] = h

### Ideally we use all dates, so step through all hours and grab from datasets accordingly.
**Importantly, we should ideally use first six hours of each dataset**

In [ ]:
# print(ids[1])
# id_sets[ids[1]]

In [ ]:
# # IT'S THERE
# my_key = ('dispersion_20240209.nc', datetime.datetime(2024, 2, 10, 0, 0))
# id_sets[ids[1]][my_key]

In [ ]:
# # Array to hold the final order we will search files, tuples of dataset ID and file name
# conversion_seq = np.empty((24000, 3), dtype=object)
# # Initialize each row to ['','',0]
# conversion_seq[:] = ['', '', 0]

conversion_seq = []
idx_calls = []

# Define the start and end dates we will step through
start_date = datetime.datetime.strptime("20210304", "%Y%m%d")
end_date = datetime.datetime.strptime("20240210", "%Y%m%d")

# the dataset we wanna use
curr_id_idx = 0

# iterate over each day
current_date = start_date
while current_date <= end_date:
    # get dispersion file for current_date
    today_file_str = f'dispersion_{current_date.strftime("%Y%m%d")}.nc'
    # get dispersion file for previous date, may be needed:
    yesterday =  current_date + datetime.timedelta(days=-1)
    yest_file_str = f'dispersion_{yesterday.strftime("%Y%m%d")}.nc'

    # file string to use
    file_str = ''
    
    # if we need to use yesterday dispersion file or not
    yes_yesterday = False
    ids = ["BSC18CA12-01", "BSC00CA12-01", "BSC06CA12-01", "BSC12CA12-01"]
    # iterate over each hour of the current day
    current_hour = datetime.datetime(current_date.year, current_date.month, current_date.day)
    while current_hour < current_date + datetime.timedelta(days=1):
        # based on the hour, grab from corresponding dataset id
        if current_hour <= current_date.replace(hour=2):
            # HERE WE NEED TO USE PRIOR DATE, UPDATE FILE STR
            yes_yesterday = True
            curr_id_idx = 3 # BSC12CA12-01
        if current_hour >= current_date.replace(hour=3) and current_hour <= current_date.replace(hour=8):
            yes_yesterday = False
            curr_id_idx = 0 # BSC18CA12-01
        if current_hour >= current_date.replace(hour=9) and current_hour <= current_date.replace(hour=14):
            yes_yesterday = False
            curr_id_idx = 1 # BSC00CA12-01
        if (current_hour >= current_date.replace(hour=15) and current_hour <= current_date.replace(hour=20)):
            yes_yesterday = False
            curr_id_idx = 2 # BSC06CA12-01
        if current_hour >= current_date.replace(hour=21):
            yes_yesterday = False
            curr_id_idx = 3 # BSC12CA12-01

        # choose desired file name
        if yes_yesterday:
            file_str = yest_file_str
        else:
            file_str = today_file_str
        
        # if desired timestamp at desired dataset id is available, use it
        if (file_str, current_hour) in id_sets[ids[curr_id_idx]]:
            # append dataset id, file name, and index of time step for idx conversion
            tstep_idx = id_sets[ids[curr_id_idx]][(file_str, current_hour)]

            # set as next in conversion sequence
            conversion_seq.append([ids[curr_id_idx], file_str, tstep_idx])

            # get file's path
            path = f'{firesmoke_dir}/{ids[curr_id_idx]}/{file_str}'
            # open the file with xarray
            ds = xr.open_dataset(path)
            idx_calls.append([ids[curr_id_idx], file_str, parse_tflag(ds['TFLAG'].values[tstep_idx][0])])
        else: # search the others in order of most recent to least
            found = 0
            search_count = 0

            print(f'NOT avail in {ids[curr_id_idx]}')
            
            # ensure curr_id_idx is big enough to subract from
            curr_id_idx = curr_id_idx + 4

            # if we are using yesterday file, we now use today file
            if yes_yesterday:
                file_str = today_file_str
            
            while found == 0 and search_count < 3:
                # try previous id in sequence
                curr_id_idx = (curr_id_idx - 1) % 4
                # if timestamp is available, use it
                if (file_str, current_hour) in id_sets[ids[curr_id_idx]]:
                    # append dataset id, file name, and index of time step for idx conversion
                    tstep_idx = id_sets[ids[curr_id_idx]][(file_str, current_hour)]
                    
                    # append dataset id, file name, and index of time step for idx conversion
                    conversion_seq.append([ids[curr_id_idx], file_str, tstep_idx])

                    # get file's path
                    path = f'{firesmoke_dir}/{ids[curr_id_idx]}/{file_str}'
                    # open the file with xarray
                    ds = xr.open_dataset(path)
                    idx_calls.append([ids[curr_id_idx], file_str, parse_tflag(ds['TFLAG'].values[tstep_idx][0])])

                    found = 1
                else:  
                    search_count += 1
                print(f'findingggg, found = {found}, search_count = {search_count}')
                print(f'ids[curr_id_idx] = {ids[curr_id_idx]}')

            # if we *still* haven't found file, try a final check, check ALL datasets again but for yesterday str
            if not found:
                # use yesterday file
                file_str = yest_file_str

                # reset search counters and conditions
                found = 0
                search_count = 0

                while found == 0 and search_count < 4:
                    # try previous id in sequence
                    curr_id_idx = (curr_id_idx - 1) % 4
                    # if timestamp is available, use it
                    if (file_str, current_hour) in id_sets[ids[curr_id_idx]]:
                        # append dataset id, file name, and index of time step for idx conversion
                        tstep_idx = id_sets[ids[curr_id_idx]][(file_str, current_hour)]
                        
                        # append dataset id, file name, and index of time step for idx conversion
                        conversion_seq.append([ids[curr_id_idx], file_str, tstep_idx])
    
                        # get file's path
                        path = f'{firesmoke_dir}/{ids[curr_id_idx]}/{file_str}'
                        # open the file with xarray
                        ds = xr.open_dataset(path)
                        idx_calls.append([ids[curr_id_idx], file_str, parse_tflag(ds['TFLAG'].values[tstep_idx][0])])
    
                        found = 1
                    else:  
                        search_count += 1

                    print(f'FAILED TO FIND, LAST RESORT:')
                    print(f'found = {found}, search_count = {search_count}')
                    print(f'ids[curr_id_idx] = {ids[curr_id_idx]}')
                if not found:
                    print('FAILed TO FIND ANYTHING')
                    
        # move to next hour
        current_hour += datetime.timedelta(hours=1)

    # move to the next day
    current_date += datetime.timedelta(days=1)
    print(f'{current_date}, ids[curr_id_idx] = {ids[curr_id_idx]}')

In [ ]:
# %%capture captured_output
# for c in idx_calls:
#     print(c)

# with open('idx_calls_v3.txt', 'w') as f:
#     f.write(captured_output.stdout)

In [ ]:
# save conversion_seq to file
with open('conversion_seq_v3.pkl', 'wb') as f:
    pickle.dump(conversion_seq, f)

# save idx_calls to file
with open('idx_calls_v3.pkl', 'wb') as f:
    pickle.dump(idx_calls, f)

In [ ]:
# Load conversion_seq from a file
with open('conversion_seq_v3.pkl', 'rb') as f:
    conversion_seq = pickle.load(f)

# Load idx_calls from a file
with open('idx_calls_v3.pkl', 'rb') as f:
    idx_calls = pickle.load(f)

In [ ]:
conversion_seq[-10:]

In [ ]:
idx_calls[-10:]

## Do conversion from netCDF files to IDX

In [ ]:
idx_calls[22200:22200+145]

In [ ]:
#********** FOR TESTING
# my_seq = conversion_seq[22200:22200+145]
my_seq = conversion_seq

In [ ]:
print(f'len(conversion_seq) = {len(conversion_seq)}')
print(f'len(idx_calls) = {len(idx_calls)}')
print(f'len(my_seq) = {len(my_seq)}')

In [ ]:
# # Create idx file of i'th dataset
# # useful for dealing with fields that are not all the same size:
# # https://github.com/sci-visus/OpenVisus/blob/master/Samples/jupyter/nasa_conversion_example.ipynb
   
# # create OpenVisus field for the pm25 variable
# f = Field('PM25', 'float32')

# # create the idx file for this dataset using field f
# # dims is maximum array size, we will resample data accordingly to fit this
# # time is number of files * 24 (hours)
# db = CreateIdx(url=idx_dir + '/firesmoke.idx', fields=[f], 
#                dims=[int(max_ncols[ids[0]]), int(max_nrows[ids[0]])], time=[0, len(my_seq) - 1, '%00000000d/'])

# # to track what timestep we are on in idx
# tstep = 0

# # threshold to use to change small-enough resampled values to 0
# thresh = 1e-15

# for call in tqdm(my_seq):
#     # get instructions from call
#     curr_id = call[0]
#     curr_file = call[1]
#     tstep_index = call[2]
#     # open the file with xarray
#     ds = xr.open_dataset(f'{firesmoke_dir}/{curr_id}/{curr_file}')
    
#     # Get the PM25 values, squeeze out empty axis
#     file_vals = np.squeeze(ds['PM25'].values)
    
#     # to decide if we need to resample or not
#     resamp = ds.XORIG != max_xorig
    
#     # resample data if not already on max lat/lon grid
#     if resamp:
#         # Perform the interpolation
#         file_vals_resamp = griddata(sml_tups, file_vals[tstep_index].flatten(), big_tups, method='cubic', fill_value=0)
        
#         # Any values that are less than a given threshold, make it 0
#         file_vals_resamp[file_vals_resamp < thresh] = 0
        
#         # Reshape the result to match the new grid shape
#         file_vals_resamp = file_vals_resamp.reshape((len(big_lat), len(big_lon)))
#         # Write resampled values at hour h to timestep t and field f
#         db.write(data=file_vals_resamp.astype(np.float32),field=f,time=tstep)
#     else:
#         # Write original values at hour h to timestep t and field f
#         db.write(data=file_vals[tstep_index], field=f, time=tstep)

#     # move to next timestep in IDX
#     tstep = tstep + 1

In [ ]:
# go to idx data directory
os.chdir(idx_dir)

In [ ]:
# compress dataset
db.compressDataset(['zip'])